In [1]:
import numpy as np
import pickle
import os
from numpy.random import rand
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop
from keras.regularizers import l2

Using Theano backend.


   ## Problema 3

### a)
Definimos las funciones para extraer los diccionarios de los archivos adjuntos de la siguiente forma. El resultado son 6 matrices, las dos primeras contienen los datos y etiquetas del set de entrenamiento, de dimensiones (50000,3072) y (50000,1). Las siguientes dos corresponden a los datos y etiquetas de pruebas, de dimensiones (10000, 3072) y (10000,1). Por último, se toman 5000 imágenes del set de entrenamiento para crear el set de validación.

In [2]:
def load_batch(filename):
    with open(filename, 'rb') as fo:
        datadict = pickle.load(fo, encoding='latin1')
        X = datadict['data']
        Y = datadict['labels']
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(path):
    xs = []
    ys = []
    for batch in range(1,6):
        file = os.path.join(path, 'data_batch_%d' % (batch,))
        X, Y = load_batch(file)
        # Se transforma el arreglo de forma que coincida con la representación original de la imágen.
        xs.append(X.astype('float'))
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X,Y
    Xte, Yte = load_batch(os.path.join(path, 'test_batch'))
    # El set de validación (5000 imágenes) extraído del set de entrenamiento.
    Xva, Yva = Xtr[15000:20000], Ytr[15000:20000] 
    return Xtr, Ytr, Xte, Yte, Xva, Yva

Xtr, Ytr, Xte, Yte, Xva, Yva = load_CIFAR10('./')
    

### b)
En primera instancia, centraremos restando la media, para luego escalar dividiendo por la desviación estándar, proceso conocido como normalización. Utilizamos el StandardScaler que realiza precisamente esta operación.

In [22]:
def normalize_data(data):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler().fit(data)
    scaled_data = scaler.transform(data)
    return scaled_data

Xtr_normalized = normalize_data(Xtr)
print('Mínimo: %f' % np.min(Xtr_normalized))
print('Máximo: %f' % np.max(Xtr_normalized))
print('Media: %f' % np.mean(Xtr_normalized))

Mínimo: -2.207429
Máximo: 2.625075
Media: 0.000000


Se observa que la media es 0. 

Ya que todos las características se encuentran en la misma escala, es posible realizar sólo el centrado, restando la media. 

In [5]:
def center_data(data):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler(with_std=False).fit(data)
    scaled_data = scaler.transform(data)
    return scaled_data

Xtr_centered = center_data(Xtr)
print('Mínimo: %f' % np.min(Xtr_centered))
print('Máximo: %f' % np.max(Xtr_centered))
print('Media: %f' % np.mean(Xtr_centered))

Mínimo: -140.268820
Máximo: 155.004940
Media: 0


Con el centrado se obtiene una media igual a 0, pero en una escala mayor. 

También es posible realizar un escalado MinMax, que entrega un set de datos entre 0 y 1. Como el mínimo en el espacio RGB es 0 y el máximo 255, es posible simplificar la función, y sólo dividir por 255. No es exactamente equivalente al escalado MinMax, pero para este caso es útil. 

In [3]:
def minmax_data(data):
    return data/255

Xtr_minmax = minmax_data(Xtr)
print('Mínimo: %f' % np.min(Xtr_minmax))
print('Maximo: %f' % np.max(Xtr_minmax))
print('Media: %f' % np.mean(Xtr_minmax))

Xva_minmax = minmax_data(Xva)
Xte_minmax = minmax_data(Xte)

Mínimo: 0.000000
Maximo: 1.000000
Media: 0.473363


### c)
Como se indica en el enunciado, comenzaremos con una red pequeña, con una capa oculta de 50 neuronas y funciones de activación ReLu. Como se trata de un problema de clasificación, todas las redes utilizadas tendrán salida softmax.

In [11]:
# Crear el modelo
small_model = Sequential()
small_model.add(Dense(50, input_dim=3072, activation='relu'))
small_model.add(Dense(10, activation='softmax'))
# Compilar modelo
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)
small_model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entrenar modelo
small_model.fit(Xtr_minmax, Ytr, nb_epoch=50)



Epoch 1/50
50000/50000 [==============================] - 13s - loss: 2.0638 - acc: 0.2125    
Epoch 2/50
50000/50000 [==============================] - 21s - loss: 1.9843 - acc: 0.2466    
Epoch 3/50
50000/50000 [==============================] - 20s - loss: 1.9467 - acc: 0.2659    
Epoch 4/50
50000/50000 [==============================] - 22s - loss: 1.9338 - acc: 0.2710    
Epoch 5/50
50000/50000 [==============================] - 22s - loss: 1.9199 - acc: 0.2760    
Epoch 6/50
50000/50000 [==============================] - 21s - loss: 1.9038 - acc: 0.2898    
Epoch 7/50
50000/50000 [==============================] - 22s - loss: 1.8849 - acc: 0.2930    
Epoch 8/50
50000/50000 [==============================] - 24s - loss: 1.8620 - acc: 0.3080    
Epoch 9/50
50000/50000 [==============================] - 23s - loss: 1.8447 - acc: 0.3162    
Epoch 10/50
50000/50000 [==============================] - 24s - loss: 1.8255 - acc: 0.3266    
Epoch 11/50
50000/50000 [========================

In [12]:
sm_scores = small_model.evaluate(Xva_minmax, Yva)
print("\nPrecisión de clasificación: {0:.2f}%".format(sm_scores[1]*100))

4960/5000 [============================>.] - ETA: 0s
Precisión de clasificación: 40.12%


Para el modelo simple, se logra un 40.12% de precisión luego de 50 epochs, utilizando backpropagation con tasa de aprendizaje decreciente (con momentum) y sin *weight decay*.

En el siguiente experimento, modelaremos un *Multilayer Perceptron (MLP)*, que consiste de dos capas ocultas de 64 neuronas cada una, con activación ReLu. Ambas capas tendrán un dropout de 0.5. El entrenamiento se realizará en mini-batches y tasa de aprendizaje decreciente con momentum. 

In [30]:
# Crear el modelo
medium_model = Sequential()
medium_model.add(Dense(64, input_dim=3072, init='uniform'))
medium_model.add(Activation('tanh'))
medium_model.add(Dense(32, init='uniform'))
medium_model.add(Activation('tanh'))
medium_model.add(Dense(10, init='uniform'))
medium_model.add(Activation('softmax'))
# Compilar modelo
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
medium_model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entrenar modelo
medium_model.fit(Xtr_minmax, Ytr, nb_epoch=10, batch_size=16)

INFO (theano.gof.compilelock): Refreshing lock /home/diego/.theano/compiledir_Linux-3.19--generic-x86_64-with-elementary_OS-0.3.2-freya-x86_64-3.4.3-64/lock_dir/lock


Epoch 1/10
50000/50000 [==============================] - 19s - loss: 2.5142 - acc: 0.1061    
Epoch 2/10
50000/50000 [==============================] - 25s - loss: 2.5402 - acc: 0.1013    
Epoch 3/10
50000/50000 [==============================] - 29s - loss: 2.5407 - acc: 0.1019    
Epoch 4/10
50000/50000 [==============================] - 24s - loss: 2.5432 - acc: 0.0998    
Epoch 5/10
50000/50000 [==============================] - 22s - loss: 2.5433 - acc: 0.1014    
Epoch 6/10
50000/50000 [==============================] - 27s - loss: 2.5378 - acc: 0.1004    
Epoch 7/10
50000/50000 [==============================] - 32s - loss: 2.5435 - acc: 0.1004    
Epoch 8/10
50000/50000 [==============================] - 36s - loss: 2.5349 - acc: 0.0981    
Epoch 9/10
50000/50000 [==============================] - 34s - loss: 2.5412 - acc: 0.0995    
Epoch 10/10
50000/50000 [==============================] - 37s - loss: 2.5359 - acc: 0.0982    


In [31]:
md_scores = medium_model.evaluate(Xva_minmax, Yva)
print("\nPrecisión de clasificación: {0:.2f}%".format(md_scores[1]*100))

4832/5000 [===========================>..] - ETA: 0s
Precisión de clasificación: 10.02%


Vemos que el nuevo modelo obtuvo peores resultados que el anterior. EL MLP modelado es útil para la clasificación multiclase, que es lo que se pretende en este ejercicio, pero queda claro que la primera capa no es capaz de extraer las características necesarias de forma correcta, por lo que la segunda capa trabaja con un input que no representativo de la imágen.

El problema con las capas utilizadas, es que no consideran la información espacial de la imágen. De la forma en que se han modelado las redes, intentan predecir la categoría de una imágen en base a los colores que la componen. Pero gran parte de sus características se encuentran en forma de bordes y contornos, en diferentes escalas. 

El siguiente modelo utiliza capas convolucionales, que aplican filtros en 'ventanas' pequeñas de la imágen, y permiten extraer características relevantes a nivel de vecindarios (o celdas), en contraste con las características a nivel de pixel de los modelos anteriores. El modelo consta de cuatro capas convolucionales, las dos primeras realizarán 32 filtros de 3x3 y las dos últimas 64 filtros de 3x3. Entre las parejas de capas existe una capa de pooling, que realiza un downsampling de factor 2, con dropout de 0.25. Luego de las capas convolucionales, la imágen se 'aplana' (vector unidimensional) y se pasa a una capa totalmente conectada de 512 neuronas con dropout 0.5 para finalmente llegar a la capa de salida, con 10 neuronas y activación softmax. Todas las otras capas utilizan activación ReLu.

Es necesario transformar la matriz de datos Xtr de forma que las imágenes queden con dimensiones (3,32,32).

In [6]:
# Transformar los conjuntos de datos
Xtr_orig = Xtr_minmax.reshape(50000,3,32,32).transpose(0,1,3,2)
Xva_orig = Xva_minmax.reshape(5000,3,32,32).transpose(0,1,3,2)
Xte_orig = Xte_minmax.reshape(10000,3,32,32).transpose(0,1,3,2)

In [5]:
# Crear el modelo
conv_model = Sequential();
conv_model.add(Convolution2D(32,3,3, input_shape=(3,32,32), border_mode='same'))
conv_model.add(Activation('relu'))
conv_model.add(Convolution2D(32,3,3))
conv_model.add(Activation('relu'))
conv_model.add(MaxPooling2D(pool_size=(2,2)))
conv_model.add(Dropout(0.25))

conv_model.add(Convolution2D(64,3,3, border_mode='same'))
conv_model.add(Activation('relu'))
conv_model.add(Convolution2D(64,3,3))
conv_model.add(Activation('relu'))
conv_model.add(MaxPooling2D(pool_size=(2,2)))
conv_model.add(Dropout(0.25))

conv_model.add(Flatten())
conv_model.add(Dense(512))
conv_model.add(Activation('relu'))
conv_model.add(Dropout(0.5))
conv_model.add(Dense(10))
conv_model.add(Activation('softmax'))

# Compilar el modelo
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
conv_model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

conv_model.fit(Xtr_orig, Ytr, nb_epoch=5, batch_size=32)

Epoch 1/5
50000/50000 [==============================] - 484s - loss: 1.7078 - acc: 0.3738   
Epoch 2/5
50000/50000 [==============================] - 456s - loss: 1.2789 - acc: 0.5417   
Epoch 3/5
50000/50000 [==============================] - 460s - loss: 1.1097 - acc: 0.6066   
Epoch 4/5
50000/50000 [==============================] - 441s - loss: 0.9976 - acc: 0.6487   
Epoch 5/5
50000/50000 [==============================] - 446s - loss: 0.9311 - acc: 0.6712   


Los resultados parecen indicar que este tipo de redes es capaz de extraer características relevantes de las imágenes, que permiten obtener una precisión alta de clasificación.

In [9]:
conv_scores = conv_model.evaluate(Xva_orig, Yva)
print("\nPrecisión de clasificación: {0:.2f}%".format(conv_scores[1]*100))

5000/5000 [==============================] - 13s    

Precisión de clasificación: 74.20%


Como el modelo convolucional obtuvo el mejor resultado, evaluamos el conjunto de pruebas.

In [ ]:
test_scores = conv_model.evaluate(Xte_orig, Yte)
print("\nPrecisión de clasificación: {0:.2f}%".format(test_scores[1]*100))

La precisión del modelo sobre el conjunto de pruebas, es de 68.40%, superando el 50% indicado en el enunciado.

### d)
Importamos la librería adjunta a la tarea. Debido a que el modelo convolucional requiere de un input de dimensiones (50000,3,32,32) y el extractor de características entrega vectores unidimensionales, haremos la comparación con el modelo simple, que obtuvo 40.12% de precisión.

In [4]:
from top_level_features import hog_features
from top_level_features import color_histogram_hsv
from top_level_features import extract_features

In [7]:
features_hog = extract_features(Xtr,[hog_features])
features_hog.shape
features_hsv = extract_features(Xtr,[color_histogram_hsv])
features_hsv.shape


(50000, 32, 32, 3)


/home/diego/dev/ANN/Tarea1/tarea1-ann/top_level_features.py:123: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  orientation_histogram[:,:,i] = uniform_filter(temp_mag, size=(cx, cy))[cx/2::cx, cy/2::cy].T


(50000, 32, 32, 3)


(50000, 10)

Para los descriptores HOG, ajustamos la capa de input a la dimensión de los vectores.

In [8]:
# Crear el modelo
small_model_hog = Sequential()
small_model_hog.add(Dense(50, input_dim=features_hog.shape[1], activation='relu'))
small_model_hog.add(Dense(10, activation='softmax'))
# Compilar modelo
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)
small_model_hog.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entrenar modelo
small_model_hog.fit(features_hog, Ytr, nb_epoch=50)

Epoch 1/50
50000/50000 [==============================] - 2s - loss: 1.6726 - acc: 0.4114     
Epoch 2/50
50000/50000 [==============================] - 2s - loss: 1.5328 - acc: 0.4678     
Epoch 3/50
50000/50000 [==============================] - 2s - loss: 1.4900 - acc: 0.4804     
Epoch 4/50
50000/50000 [==============================] - 2s - loss: 1.4704 - acc: 0.4907     
Epoch 5/50
50000/50000 [==============================] - 2s - loss: 1.4500 - acc: 0.4964     
Epoch 6/50
50000/50000 [==============================] - 2s - loss: 1.4367 - acc: 0.5052     
Epoch 7/50
50000/50000 [==============================] - 2s - loss: 1.4191 - acc: 0.5116     
Epoch 8/50
50000/50000 [==============================] - 2s - loss: 1.4055 - acc: 0.5148     
Epoch 9/50
50000/50000 [==============================] - 2s - loss: 1.3993 - acc: 0.5168     
Epoch 10/50
50000/50000 [==============================] - 2s - loss: 1.3864 - acc: 0.5246     
Epoch 11/50
50000/50000 [========================

In [9]:
features_test_hog = extract_features(Xte,[hog_features])
sm_hog_scores = small_model_hog.evaluate(features_test_hog, Yte)
print("\nPrecisión de clasificación: {0:.2f}%".format(sm_hog_scores[1]*100))

(10000, 32, 32, 3)


/home/diego/dev/ANN/Tarea1/tarea1-ann/top_level_features.py:123: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  orientation_histogram[:,:,i] = uniform_filter(temp_mag, size=(cx, cy))[cx/2::cx, cy/2::cy].T


 9536/10000 [===========================>..] - ETA: 0s
Precisión de clasificación: 49.70%


El resultado muestra un incremento de 9.58% utilizando los descriptores HOG, evaluando contra el conjunto de pruebas. Realizamos el mismo procedimiento para los histogramas HSV.

In [10]:
# Crear el modelo
small_model_hsv = Sequential()
small_model_hsv.add(Dense(50, input_dim=features_hsv.shape[1], activation='relu'))
small_model_hsv.add(Dense(10, activation='softmax'))
# Compilar modelo
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)
small_model_hsv.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entrenar modelo
small_model_hsv.fit(features_hsv, Ytr, nb_epoch=50)

Epoch 1/50
50000/50000 [==============================] - 2s - loss: 2.1152 - acc: 0.2087     
Epoch 2/50
50000/50000 [==============================] - 2s - loss: 2.0490 - acc: 0.2348     
Epoch 3/50
50000/50000 [==============================] - 1s - loss: 2.0328 - acc: 0.2400     
Epoch 4/50
50000/50000 [==============================] - 2s - loss: 2.0222 - acc: 0.2479     
Epoch 5/50
50000/50000 [==============================] - 2s - loss: 2.0140 - acc: 0.2517     
Epoch 6/50
50000/50000 [==============================] - 2s - loss: 2.0085 - acc: 0.2538     
Epoch 7/50
50000/50000 [==============================] - 2s - loss: 2.0035 - acc: 0.2573     
Epoch 8/50
50000/50000 [==============================] - 2s - loss: 2.0002 - acc: 0.2580     
Epoch 9/50
50000/50000 [==============================] - 2s - loss: 1.9973 - acc: 0.2592     
Epoch 10/50
50000/50000 [==============================] - 2s - loss: 1.9947 - acc: 0.2618     
Epoch 11/50
50000/50000 [========================

In [11]:
features_test_hsv = extract_features(Xte,[color_histogram_hsv])
sm_hsv_scores = small_model_hsv.evaluate(features_test_hsv, Yte)
print("\nPrecisión de clasificación: {0:.2f}%".format(sm_hsv_scores[1]*100))

(10000, 32, 32, 3)
 9312/10000 [==========================>...] - ETA: 0s
Precisión de clasificación: 27.21%


Se observa que la función de pérdida no varía mucho en cada iteración, por lo que es posible que se encuentre en un óptimo local. El resultado empeoró al usar los histogramas, disminuyendo la precisión en un 12.90%.

Por último, repetimos el experimento para ambos descriptores en conjunto.

In [12]:
features_hog_hsv = extract_features(Xtr,[hog_features, color_histogram_hsv])
features_test_hog_hsv = extract_features(Xte,[hog_features, color_histogram_hsv])
# Crear el modelo
small_model_hog_hsv = Sequential()
small_model_hog_hsv.add(Dense(50, input_dim=features_hog_hsv.shape[1], activation='relu'))
small_model_hog_hsv.add(Dense(10, activation='softmax'))
# Compilar modelo
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)
small_model_hog_hsv.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entrenar modelo
small_model_hog_hsv.fit(features_hog_hsv, Ytr, nb_epoch=50)

(50000, 32, 32, 3)


/home/diego/dev/ANN/Tarea1/tarea1-ann/top_level_features.py:123: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  orientation_histogram[:,:,i] = uniform_filter(temp_mag, size=(cx, cy))[cx/2::cx, cy/2::cy].T


(10000, 32, 32, 3)
Epoch 1/50
50000/50000 [==============================] - 1s - loss: 1.6483 - acc: 0.4195     
Epoch 2/50
50000/50000 [==============================] - 1s - loss: 1.4807 - acc: 0.4840     
Epoch 3/50
50000/50000 [==============================] - 1s - loss: 1.4368 - acc: 0.5009     
Epoch 4/50
50000/50000 [==============================] - 1s - loss: 1.4056 - acc: 0.5116     
Epoch 5/50
50000/50000 [==============================] - 2s - loss: 1.3842 - acc: 0.5201     
Epoch 6/50
50000/50000 [==============================] - 1s - loss: 1.3667 - acc: 0.5240     
Epoch 7/50
50000/50000 [==============================] - 2s - loss: 1.3490 - acc: 0.5303     
Epoch 8/50
50000/50000 [==============================] - 2s - loss: 1.3396 - acc: 0.5321     
Epoch 9/50
50000/50000 [==============================] - 2s - loss: 1.3307 - acc: 0.5345     
Epoch 10/50
50000/50000 [==============================] - 2s - loss: 1.3186 - acc: 0.5375     
Epoch 11/50
50000/50000 [=====

In [13]:
sm_hog_hsv_scores = small_model_hog_hsv.evaluate(features_test_hog_hsv, Yte)
print("\nPrecisión de clasificación: {0:.2f}%".format(sm_hog_hsv_scores[1]*100))

 9440/10000 [===========================>..] - ETA: 0s
Precisión de clasificación: 51.61%


Finalmente, vemos que al utilizar en conjunto los descriptores, se obtiene un aumento de 11.49% en precisión. Se puede concluir que los histogramas de colores contienen información relevante sobre las características, pero que por sí solas no son suficientes para lograr una precisión alta de clasificación. 